In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from AutoReasoner import *
import dotenv
import random

config = dotenv.dotenv_values(".env")
openai.api_type = 'azure'
openai.api_base = 'https://meta-prompter-az-openai.openai.azure.com'
openai.api_version = '2022-12-01'
openai.api_key = config['OPENAI_API_KEY_ms']

# dataset = pd.read_csv('./dataset/Table-Fact-Checking/small_test.csv', sep=',')
# dataset = pd.read_csv('./dataset/Table-Fact-Checking/dev.csv', sep=',')
# dataset = pd.read_csv('./dataset/Table-Fact-Checking/small_test_sample10.csv', sep=',')
dataset = pd.read_csv('./dataset/Table-Fact-Checking/small_test_sample10-valid.csv', sep=',')

# dataset = pd.read_csv('./dataset/Table-Fact-Checking/train_sample.csv', sep=',')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')

ft=None

In [3]:
# sampled_df = dataset.sample(frac=0.1, random_state=0)
# print(sampled_df.shape)
# sampled_df.to_csv('./dataset/Table-Fact-Checking/small_test_sample10-valid.csv', index=False)

In [4]:
# import fasttext
# ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

In [5]:
NNDemo = False
# max_demo = 5
max_demo = 8
# template = 'original-sql'
# template = 'original-sql-py-no-intermediate'
template = 'original-sql-py-no-intermediate-v1'
# template = 'formatv1'

# demo_file = f'few-shot-demo/TabFact-formatv1.json'
# demo_file = f'few-shot-demo/TabFact-formatv2.json'
# demo_file = f'few-shot-demo/TabFact-formatv3.json'
demo_file = f'few-shot-demo/TabFact-formatv4.json'
# demo_file = f'../WikiTableQuestions/few-shot-demo/WikiTQ-sql-py.json'

demo_ids = None
# demo_ids =  [21, 12, 19, 15, 2, 1, 11]  large: 80.7
# demo_ids = [2, 10, 15, 17, 9, 13, 4] # small: 83.5
# demo_ids = [13, 15, 4, 14, 1, 7, 8] # small: 85.5 

gpt_model = 'mp-aoi-codex'


def parallel_codex_func(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_template(
                                              f'./prompt_template/{template}.json',
                                              # '/mnt/text2sql/dataset/Table-Fact-Checking/prompt_template/formatv1.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              './data/all_csv/' + dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'], 
                                              base_path='./dataset/Table-Fact-Checking/',
                                              demo_file=demo_file,
                                             sep='#'
                                             )
            codex_prompter.temperature = 0
            codex_prompter.model = gpt_model
            codex_prompter.max_demo = max_demo
            codex_prompter.demo_ids = demo_ids
            # codex_prompter.demo_ids = [8, 9, 10, 11 14]
            
            codex_prompter._gen_gpt_prompt(maintain_df_ids=True)
            codex_prompter._get_gpt_prediction(maintain_df_ids=True)
            
            # codex_prompter._gen_gpt_prompt()
            # codex_prompter._get_gpt_prediction()
            
            
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            max_retry -= 1
    return log

for program in [ 'sql-py' ]:
    trials = 1
    while trials:
        trials -= 1
        # demo_ids = random.sample(range(20), 7)
        print("Using demo_ids = ", demo_ids)
        n_threads = 2
        maxLimit = float('inf')
        maxLimit = 100
        from joblib import Parallel, delayed
        logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
        json.dump(logs, open(f'./dataset/Table-Fact-Checking/results/CodexAnswerCOTExecutor_{template}_{program}_NNDemo={NNDemo}_results_test_small_limit{maxLimit}_model{gpt_model}_demo{demo_file.split("/")[-1].replace(".json", "")}.json', 'w'), indent=4)
        correct_cnt = 0
        for l in logs:
            if 'predicted_value' in l and l['target_value'] == l['predicted_value']:
                correct_cnt += 1
        print(f"Acc = {correct_cnt} / {len(logs)} = {correct_cnt / len(logs)}")
    

Using demo_ids =  None


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:03<00:00,  1.24s/it]


Acc = 79 / 100 = 0.79


In [7]:
# logs = json.load(open(f'./dataset/Table-Fact-Checking/results/CodexAnswerCOTExecutor_{template}_{program}_NNDemo={NNDemo}_results_test_small_limit{maxLimit}_model{gpt_model}.json', 'r'))
# logs = json.load(open(f'./dataset/Table-Fact-Checking/results/CodexAnswerCOTExecutor_original-sql-py-no-intermediate-v1_sql-py_NNDemo=False_results_test_small_limitinf_modelmp-aoi-codex.json', 'r'))

correct_cnt = 0
error_idx = []
for idx, l in enumerate(logs):
    if 'predicted_value' in l and l['target_value'] == l['predicted_value']:
        correct_cnt += 1
    else:
        error_idx.append(idx)

print(f"Acc = {correct_cnt} / {len(logs)} = {correct_cnt / len(logs)}")

Acc = 79 / 100 = 0.79


In [10]:
error_id = random.choice(error_idx)
print(error_id)
print(logs[error_id]['prompt'])
print(logs[error_id])
# print(logs[4]['predicted_value'])

33
The database table DF0 is shown as follows:
[HEAD]: date|opponent|score|loss|attendance|record
---
[ROW] 1: august 1|pirates|7 - 6|rincón (4 - 5)|22657|52 - 58
[ROW] 2: august 2|pirates|6 - 5|swift (4 - 5)|32388|52 - 59
[ROW] 3: august 3|pirates|8 - 4|reed (3 - 5)|24989|52 - 60
[ROW] 4: august 4|phillies|7 - 3|castillo (8 - 10)|15230|52 - 61
[ROW] 5: august 5|phillies|4 - 2|bottalico (2 - 4)|16428|53 - 61
[ROW] 6: august 6|mets|4 - 0|mlicki (5 - 8)|26633|54 - 61
[ROW] 7: august 7|mets|12 - 4|swift (4 - 6)|29536|54 - 62
[ROW] 8: august 8|pirates|5 - 3|lieber (6 - 12)|48262|55 - 62
...
[ROW] 28: august 30|athletics|4 - 3|mohler (1 - 10)|48308|67 - 70
[ROW] 29: august 31|athletics|10 - 4|oquist (2 - 5)|48041|68 - 70

Judge whether the following statement is a fact or not based on the data above: "on august 25 , remlinger (6 - 6) take the win against the rockies. yes or no?". Generate SQL or Python code step-by-step given the question and table to verify the statement correctly. For eac

In [13]:
logs[1988]

{'id': 'nt-15792',
 'utterance': 'vctp 24hp / t (23hp / ton) have a vca 155 of 155 mm (6.1in) l / 41 howitzer , and not 55 km / h (34 mph). yes or no?',
 'source_csv': './dataset/Table-Fact-Checking/./data/all_csv/2-1111973-1.html.csv',
 'target_value': 'no',
 'predicted_value': 'yes',
 'prompt': 'The database table DF0 is shown as follows:\n[HEAD]: opposing_team|against|date|venue|round\n---\n[ROW] 1: gillingham|1|1991 - aug - 20|fratton park|1st round 1st leg\n[ROW] 2: gillingham|3|1991 - aug - 27|priestfield stadium|1st round 2nd leg\n[ROW] 3: oxford united|0|1991 - sep - 24|fratton park|2nd round 1st leg\n[ROW] 4: oxford united|0|1991 - oct - 09|manor ground|2nd round 2nd leg\n[ROW] 5: manchester united|3|1991 - oct - 30|old trafford|3rd round\n\nJudge whether the following statement is a fact or not based on the data above: "there be 1 game play oppose to gillingham in august. yes or no?". Generate SQL or Python code step-by-step given the question and table to verify the statemen

In [14]:
NNDemo = False
# max_demo = 5
max_demo = 8
# template = 'original-sql'
# template = 'original-sql-py-no-intermediate'
template = 'original-sql-py-no-intermediate-v1'
# template = 'formatv1'

# demo_file = f'few-shot-demo/TabFact-formatv1.json'
# demo_file = f'few-shot-demo/TabFact-formatv2.json'
# demo_file = f'few-shot-demo/TabFact-formatv3.json'
demo_file = f'few-shot-demo/TabFact-formatv4.json'
# demo_file = f'../WikiTableQuestions/few-shot-demo/WikiTQ-sql-py.json'

gpt_model = 'mp-aoi-codex'


def parallel_codex_func(i):

    codex_prompter = CodexAnswerCOTExecutor_template(
                                      f'/mnt/GPT-TabQA/dataset/Table-Fact-Checking/prompt_template/{template}.json',
                                      # '/mnt/text2sql/dataset/Table-Fact-Checking/prompt_template/formatv1.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      './data/all_csv/' + dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'], 
                                      base_path='./dataset/Table-Fact-Checking/',
                                      demo_file=demo_file,
                                     sep='#'
                                     )
    codex_prompter.temperature = 0
    codex_prompter.model = gpt_model
    codex_prompter.max_demo = max_demo
    # codex_prompter.demo_ids = [0, 1, 2, 3, 8, 10, 14]
    # codex_prompter.demo_ids = [0, 1, 2, 3, 4, 8, 14]
    
    codex_prompter._gen_gpt_prompt(maintain_df_ids=True)
    codex_prompter._get_gpt_prediction(maintain_df_ids=True)
    log = codex_prompter._log_dict()
    return log
log = parallel_codex_func(47)

In [15]:
# print(logs[7]['prompt'])
print(log['prompt'])

The database table DF0 is shown as follows:
[HEAD]: date|opponent|score|loss|attendance|record
---
[ROW] 1: august 1|pirates|7 - 6|rincón (4 - 5)|22657|52 - 58
[ROW] 2: august 2|pirates|6 - 5|swift (4 - 5)|32388|52 - 59
[ROW] 3: august 3|pirates|8 - 4|reed (3 - 5)|24989|52 - 60
[ROW] 4: august 4|phillies|7 - 3|castillo (8 - 10)|15230|52 - 61
[ROW] 5: august 5|phillies|4 - 2|bottalico (2 - 4)|16428|53 - 61
[ROW] 6: august 6|mets|4 - 0|mlicki (5 - 8)|26633|54 - 61
[ROW] 7: august 7|mets|12 - 4|swift (4 - 6)|29536|54 - 62
[ROW] 8: august 8|pirates|5 - 3|lieber (6 - 12)|48262|55 - 62
...
[ROW] 28: august 30|athletics|4 - 3|mohler (1 - 10)|48308|67 - 70
[ROW] 29: august 31|athletics|10 - 4|oquist (2 - 5)|48041|68 - 70

Judge whether the following statement is a fact or not based on the data above: "on august 25 , remlinger (6 - 6) take the win against the rockies. yes or no?". Generate SQL or Python code step-by-step given the question and table to verify the statement correctly. For each s

In [16]:
log

{'id': 'nt-16112',
 'utterance': 'in 2011 , the southwest region win a total of 5 game and lose 1 game create a record of 28 - 12. yes or no?',
 'source_csv': './dataset/Table-Fact-Checking/./data/all_csv/2-14609295-5.html.csv',
 'target_value': 'yes',
 'predicted_value': 'yes',
 'prompt': 'The database table DF0 is shown as follows:\n[HEAD]: date|opponent|score|loss|attendance|record\n---\n[ROW] 1: august 1|pirates|7 - 6|rincón (4 - 5)|22657|52 - 58\n[ROW] 2: august 2|pirates|6 - 5|swift (4 - 5)|32388|52 - 59\n[ROW] 3: august 3|pirates|8 - 4|reed (3 - 5)|24989|52 - 60\n[ROW] 4: august 4|phillies|7 - 3|castillo (8 - 10)|15230|52 - 61\n[ROW] 5: august 5|phillies|4 - 2|bottalico (2 - 4)|16428|53 - 61\n[ROW] 6: august 6|mets|4 - 0|mlicki (5 - 8)|26633|54 - 61\n[ROW] 7: august 7|mets|12 - 4|swift (4 - 6)|29536|54 - 62\n[ROW] 8: august 8|pirates|5 - 3|lieber (6 - 12)|48262|55 - 62\n...\n[ROW] 28: august 30|athletics|4 - 3|mohler (1 - 10)|48308|67 - 70\n[ROW] 29: august 31|athletics|10 - 4|o